In [ ]:
!rm /opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info -rdf
!pip install rouge_score evaluate transformers[torch] 'accelerate>=0.26.0' -U
import torch
import numpy as np

import nltk

import transformers
from datasets import load_dataset
import evaluate
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 159.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=12228fd2c205b25b372395b4eca93757156ab943e6f1ba3f713fb745f0bb674d
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate

'cuda'

In [ ]:
'''@misc{alex2019multinews,
    title={Multi-News: a Large-Scale Multi-Document Summarization Dataset and Abstractive Hierarchical Model},
    author={Alexander R. Fabbri and Irene Li and Tianwei She and Suyi Li and Dragomir R. Radev},
    year={2019},
    eprint={1906.01749},
    archivePrefix={arXiv},
    primaryClass={cs.CL}
}'''


ds = load_dataset("Awesome075/multi_news_parquet") # This is the same to original Multi-News dataset, it is repackaged to be loaded in the easy way

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM
import torch, numpy as np

print("numpy:", np.__version__)

MODEL_ID = "mercuryujia/bart-large-multi-news"

# 1. 修改 config，让 early_stopping 不为 None
config = AutoConfig.from_pretrained(MODEL_ID)
config.early_stopping = True

# 2. 慢版 tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)

# 3. 使用修改过的 config 加载模型
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID, config=config)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("BART 加载完成 on", device)

numpy: 2.0.2


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


BART 加载完成 on cuda


In [ ]:
from transformers import GenerationConfig

source_document = """
President Joe Biden met with congressional leaders at the White House on Tuesday
to discuss a new economic relief package. The meeting comes as inflation remains
high and lawmakers are divided over the size of government spending.

Meanwhile, the Federal Reserve signaled it may slow down the pace of interest
rate hikes later this year if price increases continue to cool.

Economists say the negotiations in Washington could have major consequences
for financial markets and for American households.
"""

K = 4

gen_config = GenerationConfig(
    bos_token_id=0,
    decoder_start_token_id=2,
    early_stopping=True,
    eos_token_id=[2],
    forced_bos_token_id=0,
    forced_eos_token_id=2,
    length_penalty=2.0,
    max_length=142,
    min_length=56,
    no_repeat_ngram_size=3,
    num_beams=4,
    pad_token_id=1,
)

inputs = tokenizer(
    source_document,
    max_length=1024,
    truncation=True,
    return_tensors="pt"
).to(device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        generation_config=gen_config,
        num_return_sequences=K,
    )

candidate_summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

print("=== 生成的候选摘要 ===")
for i, s in enumerate(candidate_summaries):
    print(f"[Cand {i}] {s}\n")



=== 生成的候选摘要 ===
[Cand 0] – Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the AP. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.

[Cand 1] – Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the Hill. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.

[Cand 2] – Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the AP. The meeting comes as inflation remains high, and lawmakers are divided over

In [ ]:
!pip install evaluate rouge_score

In [ ]:
import evaluate
rouge = evaluate.load("rouge")


In [ ]:
reference = "Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the AP. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool."

scores = []
for i, cand in enumerate(candidate_summaries):
    score = rouge.compute(
        predictions=[cand],
        references=[reference],
        rouge_types=["rougeL"],
        use_stemmer=True,
    )["rougeL"]
    scores.append(score)
    print(f"[Cand {i}] ROUGE-L = {score:.4f}")

[Cand 0] ROUGE-L = 1.0000
[Cand 1] ROUGE-L = 0.9839
[Cand 2] ROUGE-L = 1.0000
[Cand 3] ROUGE-L = 0.9841


In [ ]:
test_ds = ds["test"]

print("Test size:", len(test_ds))

Test size: 5622


In [ ]:
def rouge_l(pred, ref):
    res = rouge.compute(
        predictions=[pred],
        references=[ref],
        rouge_types=["rougeL"],
        use_stemmer=True,
    )
    return res["rougeL"]

In [ ]:
from tqdm import tqdm
import pandas as pd

records = []
N = 50
for idx in tqdm(range(N), desc="Processing Test Set (first 50)"):
    item = test_ds[idx]
    document = item["document"]
    reference = item["summary"]

    # Encode input
    enc = tokenizer(
        document,
        max_length=1024,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    ).to(device)

    # Generate 4 summaries
    with torch.no_grad():
        output_ids = model.generate(
            **enc,
            generation_config=gen_config,
            num_return_sequences=4,
        )

    candidates = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

    # Compute ROUGE-L for each
    rouge_scores = [rouge_l(c, reference) for c in candidates]

    records.append({
        "index": idx,
        "document": document,
        "reference": reference,
        "cand_0": candidates[0],
        "cand_1": candidates[1],
        "cand_2": candidates[2],
        "cand_3": candidates[3],
        "rougeL_0": rouge_scores[0],
        "rougeL_1": rouge_scores[1],
        "rougeL_2": rouge_scores[2],
        "rougeL_3": rouge_scores[3],
    })
df = pd.DataFrame(records)
df.to_csv("bart_multinews_candidates_and_rougel.csv", index=False)

print("\nSaved to bart_multinews_candidates_and_rougel.csv")

Processing Test Set (first 50): 100%|██████████| 50/50 [02:05<00:00,  2.52s/it]


Saved to bart_multinews_candidates_and_rougel.csv


In [ ]:
import numpy as np

# 从之前生成的 records 列表计算 best-of-4 baseline
best_rougel = [max(r["rougeL_0"], r["rougeL_1"], r["rougeL_2"], r["rougeL_3"])
               for r in records]

baseline_rl = np.mean(best_rougel)
baseline_rl

np.float64(0.22308039622855017)